# 0. Импорт и загрузка данных

In [ ]:
# Зависимости не дружат, приходится переустанавливать отдельно
%pip install --no-deps scikit-mobility==1.3.1 geopandas==1.1.0 -qq

## 0.1 Импорт

In [4]:
import osmnx as ox

from changer_migration.planter import *
from changer_migration.model import MigrationFlowModel
from changer_migration.ueqi import count_change_capacity, get_ueqi

## 0.2 Загрузка и подготовка данных

### 0.2.1 Данные из и для TownsNet

In [ ]:
# 0) Входные пути
REGION_PKL = "data/lo_region.pickle"        # pickle с Region
MATRIX_DIR = "data/provision"               # где лежат *_links.parquet
SEED = 42

region = MigrationFlowModel.from_pickle(REGION_PKL, seed=SEED)

CPU times: total: 4 s
Wall time: 4.15 s


In [ ]:
supporting_cities = gpd.read_parquet("data/1_polygons.parquet") # Опорные города ЛО
anchor_settlement = pd.read_csv('data/anchor_settlement.csv') # Города/деревни ЛО
ueqi_df = region.calculate_ueqi() # Расчет UEQI по новой формуле

CPU times: total: 141 ms
Wall time: 222 ms


### 0.2.2 Первичное обновление UEQI для Planter
потому что мы не знаем как подсчитаны старые UEQI для городов

In [ ]:
# Загрузка модели
wff = WorkForceFlows.from_pickle('data/wff_1812 new.pkl')

Class instance loaded from data/wff_1812 new.pkl
CPU times: total: 13.4 s
Wall time: 15.4 s


In [ ]:
ueqi_df[ueqi_df.town_name == 'Бокситогорск']

CPU times: total: 15.6 ms
Wall time: 4.01 ms


,town_name,population,geometry,capacity_cafe__coffee,capacity_kindergarten,capacity_school,capacity_public_transport_stop,capacity_library,capacity_parking,capacity_railway_station,...,count_car_service,count_fire_station,settlement_name,district_name,ueqi_residential,ueqi_street_networks,ueqi_green_spaces,ueqi_public_and_business_infrastructure,ueqi_social_and_leisure_infrastructure,ueqi_citywide_space
id,,,,,,,,,,,,,,,,,,,,,
121,Бокситогорск,8001,POINT (548021.065 6593118.772),250.0,1500.0,1000.0,1500.0,500.0,2500.0,1250.0,...,0,0,Бокситогорское городское поселение,Бокситогорский муниципальный район,49.994,81.24,46.869,100.0,49.994,9.374


In [ ]:
name = "Ленинградская область, Бокситогорск" # Можно узнать все города из wff.cities

# Можно взять параметры из ueqi_df[ueqi_df.town_name == 'Мурино'] ← ВАЖНО! Убедитесь, что вы используете правильное название города без области, но для wff с областью
new_params = { # Первичное обновление UEQI для Planter
    "ueqi_residential":                         49.994,
    "ueqi_street_networks":                     81.24,
    "ueqi_green_spaces":                        46.869,
    "ueqi_public_and_business_infrastructure":  100.0,
    "ueqi_social_and_leisure_infrastructure":   49.994,
    "ueqi_citywide_space":                      9.374,
}

wff.update_city_params(name, new_params)
wff.recalculate_after_update()

Updated parameters for Ленинградская область, Бокситогорск
Recalculating after updating parameters


100%|██████████| 1106/1106 [00:02<00:00, 544.46it/s]


Recalculation complete.
CPU times: total: 10.3 s
Wall time: 11.6 s


In [ ]:
# Выделение городов только по ЛО и сохранение новых значений постоянного населения
area = ox.geocode_to_gdf("Ленинградская область")
lo_cities = wff.cities.clip(area.to_crs(3857)).copy()
lo_cities['population'] = lo_cities['population'] + lo_cities['flows_in'] - lo_cities['flows_out']
lo_cities['region_city'] = lo_cities['region_city'].str.split(', ', expand=True)[1] # У TownsNet названия городов без областей, поэтому нужно удалить их из названия
lo_cities[['region_city','population']].to_csv("data/population.csv", index=False)

CPU times: total: 312 ms
Wall time: 321 ms


### 0.2.3 Обновление населения

In [ ]:
population = pd.read_csv('data/population.csv') 
region.update_population(population)

ueqi_df = region.calculate_ueqi()
region[121] # Проверим Бокситогорск

### ====== Принимаем это за базовое население ====== ###

CPU times: total: 3.31 s
Wall time: 3.54 s


town_name                                               Бокситогорск
population                                                   15849.0
geometry                 POINT (548021.0652682963 6593118.771699116)
capacity_cafe__coffee                                          250.0
capacity_kindergarten                                         1500.0
                                            ...                     
count_railway_station                                              0
count_car_service                                                  0
count_fire_station                                                 0
settlement_name                   Бокситогорское городское поселение
district_name                     Бокситогорский муниципальный район
Name: 121, Length: 95, dtype: object

# 1. Выбор города

In [ ]:
name = 'Бокситогорск'
get_ueqi(wff, name) # Просмотр нынешных значений UEQI для города

CPU times: total: 0 ns
Wall time: 9.02 ms


,region_city,ueqi_residential,ueqi_street_networks,ueqi_green_spaces,ueqi_public_and_business_infrastructure,ueqi_social_and_leisure_infrastructure,ueqi_citywide_space
106,"Ленинградская область, Бокситогорск",49.994,81.24,46.869,100.0,49.994,9.374


# 2. Изменение UEQI

In [ ]:
new_params = {
    # "ueqi_residential":                         49.994,
    # "ueqi_street_networks":                     81.24,
    "ueqi_green_spaces":                        56.869, # +10 UEQI
    # "ueqi_public_and_business_infrastructure":  100.0,
    # "ueqi_social_and_leisure_infrastructure":   49.994,
    # "ueqi_citywide_space":                      9.374,
}

# Update the city data in the DataFrame
wff.update_city_params('Ленинградская область, Бокситогорск', new_params)
wff.recalculate_after_update()

# Выделение области
area = ox.geocode_to_gdf("Ленинградская область")
highlighted_cities = wff.cities.clip(area.to_crs(3857)).copy()
highlighted_cities['population'] = highlighted_cities['population'] + highlighted_cities['flows_in'] - highlighted_cities['flows_out']
highlighted_cities['region_city'] = highlighted_cities['region_city'].str.split(', ', expand=True)[1] # У TownsNet названия городов без областей, поэтому нужно удалить их из названия
highlighted_cities[['region_city', 'population']].to_csv("data/population_new.csv", index=False)

Updated parameters for Ленинградская область, Бокситогорск
Recalculating after updating parameters


100%|██████████| 1106/1106 [00:01<00:00, 563.47it/s]


Recalculation complete.
CPU times: total: 10.2 s
Wall time: 10.4 s


In [ ]:
# Обновим население городов после изменения UEQI
population_new = pd.read_csv('data/population_new.csv') 
region.update_population(population_new)
region[121] # Проверим Бокситогорск

### ====== Принимаем это за новое население ====== ###

CPU times: total: 3.53 s
Wall time: 3.65 s


town_name                                               Бокситогорск
population                                                   20200.0
geometry                 POINT (548021.0652682963 6593118.771699116)
capacity_cafe__coffee                                          250.0
capacity_kindergarten                                         1500.0
                                            ...                     
count_railway_station                                              0
count_car_service                                                  0
count_fire_station                                                 0
settlement_name                   Бокситогорское городское поселение
district_name                     Бокситогорский муниципальный район
Name: 121, Length: 95, dtype: object

# 3. Расчет матриц переходов по новым UEQI

## 3.1 Расчет количества и добавление необходимого сервиса

In [ ]:
# Посмотим, на сколько нужно поднять
change_capacity = count_change_capacity(region.towns, name, 10)
print(f"Изменение capacity на 10% для города {name}: {change_capacity} мест")

Изменение capacity на 10% для города Бокситогорск: 2020 мест
CPU times: total: 0 ns
Wall time: 1e+03 μs


In [ ]:
updated_service = region.update_service(name, change_capacity, new_params)
print(f'Случайный обновленный сервис: {updated_service}')

print('======= Таблица для проверки новых сервисов =======')
region.services[(region.services.town.str.contains(name)) & (region.services.service_type == updated_service)]

Случайный обновленный сервис: playground
======= Таблица для проверки новых сервисов =======
CPU times: total: 297 ms
Wall time: 463 ms


,town,service_type,geometry,capacity
181,Бокситогорск,playground,POINT (547517.067 6593154.661),250
182,Бокситогорск,playground,POINT (547984.392 6593333.93),2020


## 3.2 Расчет матриц переходов

In [ ]:
region.calculate_provision([updated_service])

✔ playground      was processed
CPU times: total: 3.72 s
Wall time: 4.84 s


# 4. Расчет данных для городов

## 4.1 Чтение матриц переходов

In [ ]:
# Для сравнения карты визуально нужно запустить разделы 4.1-5 со старым и новым населением и 2 раза запустить fmap (в разных ячейках)
UPDATED = True                              # брать обновлённые из data/provision/updated

if UPDATED:
    population = pd.read_csv('data/population_new.csv') 
    region.update_population(population)
else:
    population = pd.read_csv('data/population.csv') 
    region.update_population(population)

region.load_migration_matrix(['RECREATION'], 
                            matrix_dir='data/provision',
                            use_updated=UPDATED, # Внимание! Если True, использовать новое население. Если False, использовать базовое.
                            average=True,
                            anchors=anchor_settlement)

Загрузка матриц связей:   0%|          | 0/6 [00:00<?, ?it/s]

Результат загрузки матриц связей:
{'loaded': ['playground (updated)', 'square__boulevard__forest_park (original)', 'public_space (original)', 'park (updated)', 'amusement_park (original)', 'dog_park (original)'], 'missing': [], 'errors': []}
CPU times: total: 3.52 s
Wall time: 4.16 s


In [ ]:
mobility = region.analyze_mobility(anchor_threshold=90)

CPU times: total: 2.09 s
Wall time: 2.32 s


# 5. Генерация карты миграции

In [ ]:
fmap = region.create_map()
fmap